In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob, os
import json

import MySQLdb 
import sshtunnel
from sshtunnel import SSHTunnelForwarder

#from datetime import date 
from datetime import datetime

pd.set_option('float_format', '{:f}'.format)


In [43]:
def get_holdings(start,portfolio_amount, percentile = False, grade_type='final_grade'):
    df = pd.DataFrame()
    price = pd.DataFrame()
    ieo = pd.DataFrame()
    
    conn = MySQLdb.connect(host='tokenmetrics.cxuzrhvtziar.us-east-1.rds.amazonaws.com', user='admin', passwd='WiG8Rled2cTvZ5JibJui',db='tokenmetrics')
    cursor = conn.cursor()
    
    if percentile:
        query = '''SELECT * FROM ico_ml_percentile_grade_history;'''
        data = pd.read_sql_query(query, conn)
        df = pd.concat([df,data])
        
    else:
        query = '''SELECT * FROM ico_ml_grade_history;'''
        data = pd.read_sql_query(query, conn)
        df = pd.concat([df,data])
    
    query = 'SELECT * FROM ico_price_daily_summaries where date ="'+start+'" and currency ="USD";'
    data = pd.read_sql_query(query, conn)
    price = pd.concat([price,data])
    
    query = '''SELECT * FROM icos;'''
    data = pd.read_sql_query(query, conn)
    ieo = pd.concat([ieo,data])
    conn.close() 
    
    ieos = ieo['id'][ieo['is_traded']==0].unique().tolist()
    df['date'] = pd.to_datetime(df['date'])
    df = df[df['date'] == start]
    df = df[~df['ico_id'].isin(ieos)]
    df = df.sort_values(by=grade_type, ascending=False)
    df = df[:10]
    price = price[['ico_id','close']]
    df = df[['ico_id','date',grade_type]]
    final = pd.merge(df,price,on='ico_id')
    final.columns = ['ico_id','date','Grade','Initial Price']
    
    final['Grade Weight'] = final['Grade'] / final['Grade'].sum()
    final['Holding Amount'] = final['Grade Weight'] * portfolio_amount
    final['Holding Volume'] = final['Holding Amount'] / final['Initial Price']
    return final

In [37]:
def portfolio_amount(date,final):
    price = pd.DataFrame()

    conn = MySQLdb.connect(host='tokenmetrics.cxuzrhvtziar.us-east-1.rds.amazonaws.com', user='admin', passwd='WiG8Rled2cTvZ5JibJui',db='tokenmetrics')
    cursor = conn.cursor()

    query = 'SELECT * FROM ico_price_daily_summaries where date ="'+date+'" and currency ="USD";'
    data = pd.read_sql_query(query, conn)
    price = pd.concat([price,data])

    conn.close() 
    
    price = price[['ico_id','close']]
    price.columns = ['ico_id','New Price']
    
    temp = pd.merge(final,price)
    return sum(temp['Holding Volume'] * temp['New Price'])

In [38]:
holdings(str(datetime.today().date()),10000, False ,'technical_grade')

,ico_id,date,Grade,Initial Price,Grade Weight,Holding Amount,Holding Volume
0,3731,2020-04-24,74.486298,0.000445,0.106305,1063.049815,2391238.210184
1,3426,2020-04-24,70.034248,0.115382,0.099951,999.511281,8662.622361
2,3498,2020-04-24,70.034248,0.057815,0.099951,999.511281,17288.234194
3,3526,2020-04-24,69.691780,0.060560,0.099462,994.623660,16423.693864
4,3786,2020-04-24,69.691780,0.030435,0.099462,994.623660,32679.794715
5,3616,2020-04-24,69.691780,0.008004,0.099462,994.623660,124258.936322
6,3168,2020-04-24,69.349312,0.103707,0.098974,989.736039,9543.619589
7,3733,2020-04-24,69.349312,0.310623,0.098974,989.736039,3186.288992
8,3622,2020-04-24,69.349312,0.062715,0.098974,989.736039,15781.482309
9,3469,2020-04-24,69.006851,0.111603,0.098485,984.848526,8824.533301


In [39]:
holdings(str(datetime.today().date()),10000, True ,'technical_grade')

,ico_id,date,Grade,Initial Price,Grade Weight,Holding Amount,Holding Volume
0,3731,2020-04-24,99.211075,0.000445,0.106305,1063.049862,2391238.314351
1,3426,2020-04-24,93.281219,0.115382,0.099951,999.511271,8662.622266
2,3498,2020-04-24,93.281219,0.057815,0.099951,999.511271,17288.234006
3,3526,2020-04-24,92.825073,0.060560,0.099462,994.623649,16423.693681
4,3616,2020-04-24,92.825073,0.008004,0.099462,994.623649,124258.934932
5,3786,2020-04-24,92.825073,0.030435,0.099462,994.623649,32679.794349
6,3622,2020-04-24,92.368927,0.062715,0.098974,989.736027,15781.482128
7,3733,2020-04-24,92.368927,0.310623,0.098974,989.736027,3186.288955
8,3168,2020-04-24,92.368927,0.103707,0.098974,989.736027,9543.619479
9,3455,2020-04-24,91.912796,0.062411,0.098485,984.848569,15780.067893


In [9]:
holdings(str(datetime.today().date()),10000, True ,'technical_grade')

,ico_id,date,Grade,Initial Price,Grade Weight,Holding Amount,Holding Volume
0,3731,2020-04-24,99.211075,0.000445,0.106305,1063.049862,2389068.292513
1,3426,2020-04-24,93.281219,0.115486,0.099951,999.511271,8654.849499
2,3498,2020-04-24,93.281219,0.057862,0.099951,999.511271,17273.974699
3,3526,2020-04-24,92.825073,0.060489,0.099462,994.623649,16443.011765
4,3616,2020-04-24,92.825073,0.007965,0.099462,994.623649,124874.348505
5,3786,2020-04-24,92.825073,0.025593,0.099462,994.623649,38863.356942
6,3622,2020-04-24,92.368927,0.063504,0.098974,989.736027,15585.329345
7,3733,2020-04-24,92.368927,0.310227,0.098974,989.736027,3190.363384
8,3168,2020-04-24,92.368927,0.104170,0.098974,989.736027,9501.203771
9,3455,2020-04-24,91.912796,0.062689,0.098485,984.848569,15710.086945


In [8]:
percentile_holdings(str(datetime.today().date()),10000, 'technology_grade')

,ico_id,date,Grade,Initial Price,Grade Weight,Holding Amount,Holding Volume
0,3306,2020-04-24,77.232422,188.762634,0.105738,1057.375778,5.601616
1,3375,2020-04-24,77.232422,7542.974121,0.105738,1057.375778,0.140180
2,3312,2020-04-24,76.734154,16.289230,0.105055,1050.554075,64.493782
3,3168,2020-04-24,75.239326,0.104170,0.103009,1030.088652,9888.578285
4,3211,2020-04-24,72.196480,0.440554,0.098843,988.429563,2243.603376
5,2974,2020-04-24,71.253136,0.015191,0.097551,975.514401,64216.444743
6,3720,2020-04-24,70.256592,2.467747,0.096187,961.870890,389.776913
7,3307,2020-04-24,70.256592,8.206264,0.096187,961.870890,117.211783
8,3382,2020-04-24,70.256592,82.227997,0.096187,961.870890,11.697608
9,3738,2020-04-24,69.758316,0.696166,0.095505,955.049083,1371.870271


In [7]:
percentile_holdings(str(datetime.today().date()),10000, 'fundamental_grade')

,ico_id,date,Grade,Initial Price,Grade Weight,Holding Amount,Holding Volume
0,3375,2020-04-24,88.645409,7542.974121,0.109628,1096.276490,0.145337
1,3990,2020-04-24,85.862373,1732.553589,0.106186,1061.858733,0.612887
2,3337,2020-04-24,84.086075,0.000335,0.103989,1039.891275,3101485.781461
3,2974,2020-04-24,83.638535,0.015191,0.103436,1034.356551,68089.922827
4,3381,2020-04-24,83.069458,60.518284,0.102732,1027.318789,16.975346
5,3397,2020-04-24,80.908363,12.558712,0.100059,1000.592563,79.673183
6,3306,2020-04-24,78.268990,188.762634,0.096795,967.951465,5.127876
7,3389,2020-04-24,75.472641,333.586731,0.093337,933.369063,2.797980
8,3376,2020-04-24,75.342873,239.888611,0.093176,931.764219,3.884154
9,3108,2020-04-24,73.309769,0.178015,0.090662,906.620852,5092.937621


In [44]:
def get_hold(date, holdings,percentile, grade_type, path):
    # If initial holdings present
    if pd.to_datetime(holdings['date']).dt.month[0] != datetime.today().month:      #datetime.today().date() == datetime.today().date().replace(day=1):
        amount = portfolio_amount(date, holdings)
        holdings = get_holdings(date,amount,percentile, grade_type)
        holdings.to_csv(path, index=False)
        amount = portfolio_amount(date, holdings)
        return amount / 100
    else:
        amount = portfolio_amount(date, holdings)
        return amount / 100

In [67]:
grade_type = 'fundamental_grade'

In [47]:
grade_type = 'technology_grade'

In [75]:
percentile = False

In [76]:
if percentile:
    path = 'monthly/'+'Percentile/'+grade_type+'_holdings.csv'
else:
    path = 'monthly/'+grade_type+'_holdings.csv'

In [77]:
path

'monthly/fundamental_grade_holdings.csv'

In [80]:
date = str(datetime.today().date())

if os.path.isfile(path):
    holdings = pd.read_csv(path)
    index = get_hold(date, holdings, percentile, grade_type, path)
else:
    holdings = pd.DataFrame()
    holdings = get_holdings('2020-03-05',10000, percentile, grade_type)
    holdings.to_csv(path, index=False)
    amount = portfolio_amount(date, holdings)
    index = amount / 100

In [79]:
holdings

,ico_id,date,Grade,Initial Price,Grade Weight,Holding Amount,Holding Volume
0,3335,2020-03-05,72.049278,0.235038,0.104777,1047.766745,4457.853748
1,3389,2020-03-05,70.602676,547.766785,0.102673,1026.729735,1.874392
2,3337,2020-03-05,70.386452,0.000652,0.102359,1023.585318,1569265.079048
3,3375,2020-03-05,70.048775,9075.517578,0.101867,1018.674696,0.112244
4,3333,2020-03-05,69.129517,0.928063,0.100531,1005.306511,1083.230921
5,3391,2020-03-05,67.911293,51.946095,0.098759,987.590662,19.011837
6,3315,2020-03-05,67.618874,0.050956,0.098334,983.338193,19297.732388
7,3720,2020-03-05,66.838104,3.768689,0.097198,971.983961,257.910389
8,3390,2020-03-05,66.825386,0.054276,0.097180,971.799009,17904.720229
9,3306,2020-03-05,66.235809,228.794113,0.096323,963.225170,4.210009


In [73]:
index

71.96589392615279

In [35]:
holdings = pd.read_csv('holdings.csv')

In [36]:
holdings['date'] = holdings['date'].astype('str')

In [37]:
holdings.set_index('ico_id', inplace=True)

In [39]:
holdings.columns = ['date','Grade','Initial Price','Grade Weight','Cash Value','Amount of Tokens']

In [40]:
holdings = holdings.T.to_dict()

In [41]:
holdings['Index Value'] = index

In [42]:
holdings

{3375: {'date': '2020-04-02',
  'Grade': 88.7661590576,
  'Initial Price': 6793.18359375,
  'Grade Weight': 0.12496791689453872,
  'Cash Value': 1603.8039995375314,
  'Amount of Tokens': 0.2360901891438787},
 3306: {'date': '2020-04-02',
  'Grade': 86.93409729,
  'Initial Price': 141.62702941889998,
  'Grade Weight': 0.1223886801093745,
  'Cash Value': 1570.7027814441576,
  'Amount of Tokens': 11.090416764997466},
 3720: {'date': '2020-04-02',
  'Grade': 80.1672286987,
  'Initial Price': 1.9841561316999998,
  'Grade Weight': 0.1128620600468222,
  'Cash Value': 1448.4407502118552,
  'Amount of Tokens': 730.0034140815567},
 3315: {'date': '2020-04-02',
  'Grade': 77.9245300293,
  'Initial Price': 0.031902972599999996,
  'Grade Weight': 0.10970471513168162,
  'Cash Value': 1407.9202508016324,
  'Amount of Tokens': 44131.3186847558},
 2974: {'date': '2020-04-02',
  'Grade': 75.6725921631,
  'Initial Price': 0.0115099223,
  'Grade Weight': 0.10653436297154892,
  'Cash Value': 1367.232819973

In [95]:
index

63.58179053124445

### Weekly

In [7]:
pd.to_datetime(holdings['date'].iloc[0]) + pd.to_timedelta(7,unit='days') == datetime.today().date()

False

In [151]:
pd.to_datetime(datetime.today().date()) + pd.to_timedelta(7,unit='days')

Timestamp('2020-04-24 00:00:00')

In [4]:
def get_weekly_hold(date, holdings):
    # If initial holdings present
    if pd.to_datetime(holdings['date'].iloc[0]) + pd.to_timedelta(7,unit='days') <= datetime.today().date():
        amount = portfolio_amount(date, holdings)
        holdings = get_holdings(date,amount)
        holdings.to_csv('weekly_holdings.csv', index=False)
        amount = portfolio_amount(date, holdings)
        return amount / 100
    else:
        amount = portfolio_amount(date, holdings)
        return amount / 100

In [5]:
date = str(datetime.today().date())

if os.path.isfile('weekly_holdings.csv'):
    holdings = pd.read_csv('weekly_holdings.csv')
    index = get_weekly_hold(date, holdings)
else:
    holdings = pd.DataFrame()
    holdings = get_holdings(date,10000)
    holdings.to_csv('weekly_holdings.csv', index=False)
    amount = portfolio_amount(date, holdings)
    index = amount / 100

DatabaseError: Execution failed on sql: SELECT * FROM icos;
(2013, 'Lost connection to MySQL server during query')
unable to rollback

In [ ]:
holdings

In [ ]:
index

In [76]:
holdings

,ico_id,date,Grade,Initial Price,Grade Weight,Holding Amount,Holding Volume
0,3375,2020-04-04,92.279449,6856.738281,0.123124,1231.244112,0.179567
1,3306,2020-04-04,89.995728,145.284012,0.120077,1200.773415,8.265007
2,2974,2020-04-04,86.005325,0.011778,0.114753,1147.531233,97434.004834
3,3389,2020-04-04,83.033546,311.574371,0.110788,1107.880094,3.555748
4,3391,2020-04-04,80.879372,33.192513,0.107914,1079.137887,32.511485
5,3168,2020-04-04,80.422318,0.099173,0.107304,1073.039615,10819.868182
6,3108,2020-04-04,79.903770,0.156998,0.106612,1066.120870,6790.660413
7,3382,2020-04-04,79.090508,68.624733,0.105527,1055.269860,15.377398
8,3312,2020-04-04,77.871330,13.806095,0.103900,1039.002914,75.256827


In [15]:
index

0.0

In [163]:
index

99.96949108648737

In [164]:
holdings

,ico_id,date,Grade,Initial Price,Grade Weight,Holding Amount,Holding Volume
0,2974,2020-04-17,84.048279,0.012865,0.104159,1041.585528,80964.919992
1,3312,2020-04-17,81.578331,15.862694,0.101098,1010.976193,63.732945
2,3375,2020-04-17,81.226196,7097.050781,0.100661,1006.612292,0.141835
3,3306,2020-04-17,81.096977,171.196030,0.100501,1005.010918,5.870527
4,3153,2020-04-17,80.954239,0.004247,0.100324,1003.242004,236198.750277
5,3380,2020-04-17,80.513741,0.048955,0.099778,997.783038,20381.818034
6,3389,2020-04-17,79.657616,301.065155,0.098717,987.173335,3.278936
7,3108,2020-04-17,79.316582,0.171962,0.098295,982.947002,5716.087088
8,3796,2020-04-17,79.283173,3.481468,0.098253,982.532972,282.218014
9,3720,2020-04-17,79.251198,2.393299,0.098214,982.136718,410.369482


In [75]:

if len(holdings) == 0:
    amount = 10000
    
if datetime.today().date() == datetime.today().date().replace(day=1):
    amount = portfolio_amount(date, holdings)
    holdings = get_holdings(date,amount)
    amount = portfolio_amount(date, holdings)
else:
    amount = portfolio_amount(date, holdings)

In [42]:
pd.to_datetime(datetime.today().replace(day=1))

Timestamp('2020-04-01 11:29:29.381532')

In [28]:
get_holdings('2020-04-01',20000)

,ico_id,Grade,Initial Price,Grade Weight,Holding Amount,Holding Volume
0,3375,89.872627,6583.562012,0.124973,2499.451219,0.379650
1,3306,89.345528,135.069382,0.124240,2484.792031,18.396412
2,2974,80.271942,0.011207,0.111622,2232.446183,199206.392816
3,3382,78.624916,65.765503,0.109332,2186.640675,33.249053
4,3108,76.895515,0.151352,0.106927,2138.544245,14129.632910
5,3720,76.891144,1.972870,0.106921,2138.422665,1083.914901
6,3168,76.640755,0.093092,0.106573,2131.459084,22896.365115
7,3461,75.316368,3.003168,0.104731,2094.626516,697.472283
8,3337,75.280083,0.000340,0.104681,2093.617381,6163003.800598


In [34]:
month = 2

In [35]:
date = '2020-02-01'

In [36]:
if month == pd.to_datetime(date).month:
    amount = portfolio_amount(date, holdings)
else:
    month = pd.to_datetime(date).month
    holdings = get_holdings('2020-02-01',10000)
    amount = portfolio_amount('2020-03-01', holdings)

NameError: name 'holdings' is not defined

In [32]:
final

,ico_id,Grade,Initial Price,Grade Weight,Holding Amount,Holding Volume
0,3375,89.872627,6583.562012,0.139584,837.427578,0.127200
1,3306,89.345528,135.069382,0.138766,832.516097,6.163618
2,2974,80.271942,0.011207,0.124673,747.968989,66743.021755
3,3382,78.624916,65.765503,0.122115,732.622102,11.139915
4,3108,76.895515,0.151352,0.119429,716.507654,4734.056891
5,3720,76.891144,1.972870,0.119422,716.466919,363.159810
6,3168,76.640755,0.093092,0.119033,714.133809,7671.302981
7,3461,75.316368,3.003168,0.116977,701.793256,233.684307


In [29]:
holdings = get_holdings('2020-02-01',10000)
amount = portfolio_amount('2020-03-01', holdings)
print(amount)
final = get_holdings('2020-03-01',amount)

KeyboardInterrupt: 

In [10]:
amount = new_portfolio_amount('2020-04-01', holdings)
final = get_holdings('2020-04-01',amount)

In [11]:
amount

5999.436402634399

In [17]:
from datetime import datetime

month = pd.to_datetime(datetime.today().replace(day=1)).month

In [18]:
month

4

In [135]:
holdings = get_holdings('2020-04-01',10000)
new_portfolio_amount('2020-04-10', holdings)

(7239.108998426882,
    ico_id     Grade  Initial Price  Grade Weight  Holding Amount  \
 0    3375 89.872627    6583.562012      0.139584     1395.843746   
 1    3306 89.345528     135.069382      0.138766     1387.657175   
 2    3382 78.624916      65.765503      0.122115     1221.151542   
 3    3108 76.895515       0.151352      0.119429     1194.291606   
 4    3720 76.891144       1.972870      0.119422     1194.223708   
 
    Holding Volume   New Price  
 0        0.212020 6945.841309  
 1       10.273662  159.142593  
 2       18.568269   74.858177  
 3     7890.836028    0.169952  
 4      605.323210    2.313523  )

In [137]:
holdings

,ico_id,Grade,Initial Price,Grade Weight,Holding Amount,Holding Volume
0,3375,89.872627,6583.562012,0.139584,1395.843746,0.212020
1,3306,89.345528,135.069382,0.138766,1387.657175,10.273662
2,2974,80.271942,0.011207,0.124673,1246.732090,111248.819515
3,3382,78.624916,65.765503,0.122115,1221.151542,18.568269
4,3108,76.895515,0.151352,0.119429,1194.291606,7890.836028
5,3720,76.891144,1.972870,0.119422,1194.223708,605.323210
6,3168,76.640755,0.093092,0.119033,1190.334826,12786.706060
7,3461,75.316368,3.003168,0.116977,1169.765307,389.510432


In [132]:
amount = new_portfolio_amount('2020-04-10', final)
amount

4334.0174640022915

In [129]:
final

,ico_id,Grade,Initial Price,Grade Weight,Holding Amount,Holding Volume
0,3375,89.872627,6583.562012,0.139584,837.427578,0.127200
1,3306,89.345528,135.069382,0.138766,832.516097,6.163618
2,2974,80.271942,0.011207,0.124673,747.968989,66743.021755
3,3382,78.624916,65.765503,0.122115,732.622102,11.139915
4,3108,76.895515,0.151352,0.119429,716.507654,4734.056891
5,3720,76.891144,1.972870,0.119422,716.466919,363.159810
6,3168,76.640755,0.093092,0.119033,714.133809,7671.302981
7,3461,75.316368,3.003168,0.116977,701.793256,233.684307


In [126]:
holdings

,ico_id,Grade,Initial Price,Grade Weight,Holding Amount,Holding Volume
0,3389,93.563316,558.440674,0.107184,1071.840122,1.919345
1,3381,88.837532,73.326385,0.101770,1017.702610,13.879078
2,3306,88.781059,183.787140,0.101706,1017.055670,5.533878
3,3346,86.997711,0.180690,0.099663,996.626039,5515.653213
4,3397,86.561989,18.973019,0.099163,991.634503,52.265510
5,3382,86.153290,117.542053,0.098695,986.952539,8.396591
6,3391,86.098724,66.263458,0.098633,986.327450,14.884938
7,3406,85.967422,0.006409,0.098482,984.823286,153658.884572
8,3376,85.304268,381.492615,0.097723,977.226337,2.561587
9,3333,84.657005,0.913665,0.096981,969.811444,1061.451551


In [127]:
final

,ico_id,Grade,Initial Price,Grade Weight,Holding Amount,Holding Volume
0,3375,89.470924,8564.130859,0.106146,948.712772,0.110777
1,3389,86.950684,542.939880,0.103157,921.989178,1.698142
2,3306,85.386452,218.620865,0.101301,905.402709,4.141429
3,3377,85.081047,57.962582,0.100938,902.164324,15.564599
4,3720,83.143364,3.368157,0.098640,881.617931,261.750868
5,2974,83.065872,0.019386,0.098548,880.796241,45434.562225
6,3397,82.736015,17.174589,0.098156,877.298574,51.081197
7,3312,82.476891,18.920692,0.097849,874.550922,46.221930
8,3337,82.408852,0.000609,0.097768,873.829466,1435816.760629
9,3335,82.180473,0.211127,0.097497,871.407836,4127.410434


In [ ]:
temp['Holding Volume'] * temp['New Price']

In [120]:
get_initial_holdings('2020-04-01',6358.17905)

,ico_id,Grade,Initial Price,Grade Weight,Holding Amount,Holding Volume
0,3375,89.872627,6583.562012,0.139584,887.502446,0.134806
1,3306,89.345528,135.069382,0.138766,882.297278,6.532178
2,2974,80.271942,0.011207,0.124673,792.694586,70733.991358
3,3382,78.624916,65.765503,0.122115,776.430015,11.806038
4,3108,76.895515,0.151352,0.119429,759.351987,5017.134832
5,3720,76.891144,1.972870,0.119422,759.308816,384.875335
6,3168,76.640755,0.093092,0.119033,756.836195,8130.016659
7,3461,75.316368,3.003168,0.116977,743.757727,247.657707


In [62]:
df = pd.DataFrame()
price = pd.DataFrame()

conn = MySQLdb.connect(host='tokenmetrics.cxuzrhvtziar.us-east-1.rds.amazonaws.com', user='admin', passwd='WiG8Rled2cTvZ5JibJui',db='tokenmetrics')
cursor = conn.cursor()

#cursor.execute("SELECT * FROM ianbalina.ico_price_daily_summaries;") 
#m = cursor.fetchone()
query = '''SELECT * FROM ico_ml_percentile_grade_history;'''
data = pd.read_sql_query(query, conn)
df = pd.concat([df,data])

query = 'SELECT * FROM ico_price_daily_summaries where date ="'+start+'" and currency ="USD";'
data = pd.read_sql_query(query, conn)
price = pd.concat([price,data])
    
conn.close() 

In [106]:
get_initial_holdings(start, portfolio_amount)

,ico_id,Grade,Initial Price,Grade Weight,Holding Amount,Holding Volume
0,3375,89.470924,8564.130859,0.106146,1061.464746,0.123943
1,3389,86.950684,542.939880,0.103157,1031.565125,1.899962
2,3306,85.386452,218.620865,0.101301,1013.007398,4.633626
3,3377,85.081047,57.962582,0.100938,1009.384140,17.414410
4,3720,83.143364,3.368157,0.098640,986.395864,292.859258
5,2974,83.065872,0.019386,0.098548,985.476517,50834.338390
6,3397,82.736015,17.174589,0.098156,981.563163,57.152061
7,3312,82.476891,18.920692,0.097849,978.488960,51.715283
8,3337,82.408852,0.000609,0.097768,977.681760,1606459.741254
9,3335,82.180473,0.211127,0.097497,974.972326,4617.942122


In [88]:
final

,ico_id,Grade,Initial Price,Grade Weight,Holding Amount,Holding Volume
0,3375,89.470924,8564.130859,0.106146,1061.464746,0.123943
1,3389,86.950684,542.939880,0.103157,1031.565125,1.899962
2,3306,85.386452,218.620865,0.101301,1013.007398,4.633626
3,3377,85.081047,57.962582,0.100938,1009.384140,17.414410
4,3720,83.143364,3.368157,0.098640,986.395864,292.859258
5,2974,83.065872,0.019386,0.098548,985.476517,50834.338390
6,3397,82.736015,17.174589,0.098156,981.563163,57.152061
7,3312,82.476891,18.920692,0.097849,978.488960,51.715283
8,3337,82.408852,0.000609,0.097768,977.681760,1606459.741254
9,3335,82.180473,0.211127,0.097497,974.972326,4617.942122


In [91]:
end = '2020-04-01'

price = pd.DataFrame()

conn = MySQLdb.connect(host='tokenmetrics.cxuzrhvtziar.us-east-1.rds.amazonaws.com', user='admin', passwd='WiG8Rled2cTvZ5JibJui',db='tokenmetrics')
cursor = conn.cursor()

query = 'SELECT * FROM ico_price_daily_summaries where date ="'+end+'" and currency ="USD";'
data = pd.read_sql_query(query, conn)
price = pd.concat([price,data])
    
conn.close() 

In [110]:
price = price[['ico_id','close']]
price.columns = ['ico_id','New Price']

In [112]:
temp = pd.merge(final,price)

In [117]:
temp

,ico_id,Grade,Price,Grade Weight,Holding Amount,Holding Volume,New Price
0,3375,89.470924,6583.562012,0.106146,1061.464746,0.161230,6583.562012
1,3389,86.950684,300.034637,0.103157,1031.565125,3.438153,300.034637
2,3306,85.386452,135.069382,0.101301,1013.007398,7.499904,135.069382
3,3377,85.081047,39.218925,0.100938,1009.384140,25.737170,39.218925
4,3720,83.143364,1.972870,0.098640,986.395864,499.980286,1.972870
5,2974,83.065872,0.011207,0.098548,985.476517,87936.373900,0.011207
6,3397,82.736015,11.442900,0.098156,981.563163,85.779233,11.442900
7,3312,82.476891,12.686090,0.097849,978.488960,77.130857,12.686090
8,3337,82.408852,0.000340,0.097768,977.681760,2878012.217485,0.000340
9,3335,82.180473,0.142935,0.097497,974.972326,6821.064954,0.142935


In [116]:
(temp['Holding Volume'] * temp['New Price'])

0   1061.464746
1   1031.565125
2   1013.007398
3   1009.384140
4    986.395864
5    985.476517
6    981.563163
7    978.488960
8    977.681760
9    974.972326
dtype: float64

In [108]:
price

,ico_id,close
0,3589,0.002190
1,3108,0.151352
2,3426,0.092772
3,3455,0.061351
4,3254,0.020997
...,...,...
242,3369,0.174582
243,3797,0.224135
244,3391,31.337002
245,3461,3.003168


In [95]:
final.columns = ['ico_id','Grade','Price']

In [97]:
final['Grade Weight'] = final['Grade'] / final['Grade'].sum()
final['Holding Amount'] = final['Grade Weight'] * 10000
final['Holding Volume'] = final['Holding Amount'] / final['Price']
final

,ico_id,Grade,Price,Grade Weight,Holding Amount,Holding Volume
0,3375,89.470924,6583.562012,0.106146,1061.464746,0.161230
1,3389,86.950684,300.034637,0.103157,1031.565125,3.438153
2,3306,85.386452,135.069382,0.101301,1013.007398,7.499904
3,3377,85.081047,39.218925,0.100938,1009.384140,25.737170
4,3720,83.143364,1.972870,0.098640,986.395864,499.980286
5,2974,83.065872,0.011207,0.098548,985.476517,87936.373900
6,3397,82.736015,11.442900,0.098156,981.563163,85.779233
7,3312,82.476891,12.686090,0.097849,978.488960,77.130857
8,3337,82.408852,0.000340,0.097768,977.681760,2878012.217485
9,3335,82.180473,0.142935,0.097497,974.972326,6821.064954
